<a href="https://colab.research.google.com/github/alexphil12/Energy-forcasting-UDLA/blob/main/prediction_power_lstm_iterate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn

import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy as cp
%matplotlib inline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [2]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler

In [3]:
mesure=["Fecha","Hora"];
mesure.append("Voltaje_(R)_[V]")	
mesure.append("Voltaje_(S)_[V]")	
mesure.append("Voltaje_(T)_[V]")	
mesure.append("Voltaje_(RS)_[V]")	
mesure.append("Voltaje_(ST)_[V]")	
mesure.append("Voltaje_(TR)_[V]")	
mesure.append("Corriente_R_[A]")	
mesure.append("Corriente_S_[A]")	
mesure.append("Corriente_T_[A]")	
mesure.append("Potencia_R_[VA]")	
mesure.append("Potencia_S_[VA]")	
mesure.append("Potencia_T_[VA]")	
mesure.append("Potencia_R_[W]")	
mesure.append("Potencia_S_[W]")	
mesure.append("Potencia_T_[W]")	
mesure.append("Potencia_R_[VAR]")	
mesure.append("Potencia_S_[VAR]")	
mesure.append("Potencia_T_[VAR]")	
mesure.append("Corriente_N_[A]")	
mesure.append("Frecuencia_[Hz]")	
mesure.append("not_sure_1")	
mesure.append("not_sure_2")	
mesure.append("not_sure_3")	
mesure.append("Potencia_3F_[KVA]")	
mesure.append("Potencia_3F_[W]")	
mesure.append("Potencia_3F_[VAR]")	
mesure.append("Factor de Potencia")

In [4]:
data1=pd.read_csv("/content/df_cov_continu_1_1.txt",sep=",",header=0,names=mesure)
data2=pd.read_csv("/content/df_cov_continu_1_2.txt",sep=",",header=0,names=mesure)
data3=pd.read_csv("/content/df_cov_continu_1_3.txt",sep=",",header=0,names=mesure)
data4=pd.read_csv("/content/df_cov_continu_1_4.txt",sep=",",header=0,names=mesure)

FileNotFoundError: ignored

In [ ]:
train_data_1=data1.iloc[0:round(10253*0.8),:]

test_data_1=data1.iloc[round(10253*0.8):10253,:]

train_data_2=data1.iloc[0:round(9641*0.8),:]

test_data_2=data1.iloc[round(9641*0.8):9641,:]

train_data_3=data3.iloc[0:round(8358*0.8),:]

test_data_3=data3.iloc[round(8358*0.8):8358,:]

train_data_4=data4.iloc[0:round(5363*0.8),:]

test_data_4=data4.iloc[round(5363*0.8):5363,:]

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(data1.corr(),cmap="gray")
plt.colorbar()

In [ ]:
#train_data_1.drop(columns=[mesure[0],mesure[3],mesure[4],mesure[1],mesure[5],mesure[6],mesure[7],mesure[8],mesure[9],mesure[10],mesure[14],mesure[15],mesure[16],mesure[17],mesure[18],mesure[19],mesure[23],mesure[24],mesure[25],mesure[26]],inplace=True)
#test_data_1.drop(columns=[mesure[0],mesure[3],mesure[4],mesure[1],mesure[5],mesure[6],mesure[7],mesure[8],mesure[9],mesure[10],mesure[14],mesure[15],mesure[16],mesure[17],mesure[18],mesure[19],mesure[23],mesure[24],mesure[25],mesure[26]],inplace=True)

In [ ]:
RSG_train_1 = train_data_1.iloc[:,2].values
RSG_test_1=test_data_1.iloc[:,2].values
RSG_train_2=train_data_2.iloc[:,2].values
RSG_test_2=train_data_2.iloc[:,2].values
RSG_train_3 = train_data_3.iloc[:,2].values
RSG_test_3=test_data_3.iloc[:,2].values
RSG_train_4=train_data_4.iloc[:,2].values
RSG_test_4=train_data_4.iloc[:,2].values
plt.figure(figsize=(10,10))
plt.imshow(train_data_1.corr(),cmap="gray")
plt.colorbar()

In [ ]:
scaler = MinMaxScaler()
scaler2=MinMaxScaler()
scaler3=MinMaxScaler()
scaler4=MinMaxScaler()
RSG_train_1=np.reshape(RSG_train_1,(-1,1))
RSG_test_1=np.reshape(RSG_test_1,(-1,1))
RSG_train_2=np.reshape(RSG_train_2,(-1,1))
RSG_test_2=np.reshape(RSG_test_2,(-1,1))
RSG_train_3=np.reshape(RSG_train_3,(-1,1))
RSG_test_3=np.reshape(RSG_test_3,(-1,1))
RSG_train_4=np.reshape(RSG_train_4,(-1,1))
RSG_test_4=np.reshape(RSG_test_4,(-1,1))
scaler.fit(RSG_train_1)
scaler2.fit(RSG_train_2)
scaler3.fit(RSG_train_3)
scaler4.fit(RSG_train_4)

In [ ]:
scaled_1_train = scaler.transform(RSG_train_1)
scaled_1_test = scaler.transform(RSG_test_1)
scaled_2_train=scaler2.transform(RSG_train_2)
scaled_2_test=scaler2.transform(RSG_test_2)
scaled_3_train = scaler.transform(RSG_train_3)
scaled_3_test = scaler.transform(RSG_test_3)
scaled_4_train=scaler2.transform(RSG_train_4)
scaled_4_test=scaler2.transform(RSG_test_4)

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
n_input = 20
n_features = 0
train_generator_1 = TimeseriesGenerator(scaled_1_train, scaled_1_train, length=n_input, batch_size=10)
test_generator_1=TimeseriesGenerator(scaled_1_test, scaled_1_test, length=n_input, batch_size=10)
train_generator_2 = TimeseriesGenerator(scaled_2_train, scaled_2_train, length=n_input, batch_size=10)
test_generator_2=TimeseriesGenerator(scaled_2_test, scaled_2_test, length=n_input, batch_size=10)
train_generator_3 = TimeseriesGenerator(scaled_3_train, scaled_3_train, length=n_input, batch_size=10)
test_generator_3=TimeseriesGenerator(scaled_3_test, scaled_3_test, length=n_input, batch_size=10)
train_generator_4 = TimeseriesGenerator(scaled_4_train, scaled_4_train, length=n_input, batch_size=10)
test_generator_4=TimeseriesGenerator(scaled_4_test, scaled_4_test, length=n_input, batch_size=10)

In [ ]:
X,y = train_generator_1[0]
X1,y1=test_generator_2[0]

In [ ]:
print(f'Predicción: \n {y}')
print(f'data: \n {X}')

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU
from keras.layers import Dropout


In [ ]:
from keras.layers.recurrent_v2 import LSTM
# define model
model = Sequential()
model.add(LSTM(30, input_shape=(n_input,n_features+1),activation='relu',return_sequences=False))
model.add(Dense(1, activation = "linear"))

model.compile(loss='mse', optimizer='adam')

In [ ]:
model1 = Sequential()
model1.add(LSTM(30, input_shape=(n_input,n_features+1),activation='relu',return_sequences=False))
model1.add(Dense(1, activation = "linear"))

model1.compile(loss='mse', optimizer='adam')

In [ ]:
model2 = Sequential()
model2.add(LSTM(30, input_shape=(n_input,n_features+1),activation='relu',return_sequences=False))
model2.add(Dense(1, activation = "linear"))

model2.compile(loss='mse', optimizer='adam')

In [ ]:
model3 = Sequential()
model3.add(LSTM(30, input_shape=(n_input,n_features+1),activation='relu',return_sequences=False))
model3.add(Dense(1, activation = "linear"))

model3.compile(loss='mse', optimizer='adam')

In [ ]:
model.summary()

In [ ]:
model.fit(train_generator_1,steps_per_epoch=10, epochs=10,validation_data=test_generator_1)
model1.fit(train_generator_1,steps_per_epoch=10, epochs=10,validation_data=test_generator_1)
model2.fit(train_generator_1,steps_per_epoch=10, epochs=10,validation_data=test_generator_1)
model3.fit(train_generator_1,steps_per_epoch=10, epochs=10,validation_data=test_generator_1)

In [ ]:
loss_per_epoch = model.history.history['loss']
loss_vall=model.history.history['val_loss']
plt.plot(range(len(loss_per_epoch)),loss_per_epoch,label="train-loss")
plt.plot(range(len(loss_vall)),loss_vall,label="validation-loss")
plt.legend()

In [ ]:
model1.fit(train_generator_2,steps_per_epoch=10, epochs=10,validation_data=test_generator_2)
model2.fit(train_generator_2,steps_per_epoch=10, epochs=10,validation_data=test_generator_2)
model3.fit(train_generator_2,steps_per_epoch=10, epochs=10,validation_data=test_generator_2)

In [ ]:
loss_per_epoch = model1.history.history['loss']
loss_vall=model1.history.history['val_loss']
plt.plot(range(len(loss_per_epoch)),loss_per_epoch,label="train-loss")
plt.plot(range(len(loss_vall)),loss_vall,label="validation-loss")
plt.legend()

In [ ]:
model2.fit(train_generator_3,steps_per_epoch=10, epochs=10,validation_data=test_generator_3)
model3.fit(train_generator_3,steps_per_epoch=10, epochs=10,validation_data=test_generator_3)

In [ ]:
loss_per_epoch = model2.history.history['loss']
loss_vall=model2.history.history['val_loss']
plt.plot(range(len(loss_per_epoch)),loss_per_epoch,label="train-loss")
plt.plot(range(len(loss_vall)),loss_vall,label="validation-loss")
plt.legend()

In [ ]:
model3.fit(train_generator_3,steps_per_epoch=10, epochs=10,validation_data=test_generator_3)

In [ ]:
loss_per_epoch = model3.history.history['loss']
loss_vall=model3.history.history['val_loss']
plt.plot(range(len(loss_per_epoch)),loss_per_epoch,label="train-loss")
plt.plot(range(len(loss_vall)),loss_vall,label="validation-loss")
plt.legend()

In [ ]:
test_predictions = []
Q=100
N=400
first_eval_batch = scaled_1_train[Q:Q+n_input]
current_batch = first_eval_batch.reshape((1, n_input, n_features+1))
for i in range(N):
    
    # obtener la predicción ([0] es para obtener solo el número en lugar de [matriz])
    current_pred = model.predict(current_batch)[0]
    
    # guardar la predicción
    test_predictions.append(current_pred) 
    
    # actualizar el lote para incluir ahora la predicción y soltar primer valor
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
L_true=  list(data1[mesure[3]][n_input+1+Q:n_input+1+Q+N])

In [ ]:

trainPredict = model.predict(train_generator_1)
trainPredict1= model1.predict(train_generator_1)
trainPredict2 = model2.predict(train_generator_1)
trainPredict3= model3.predict(train_generator_1)
true_pre=scaler.inverse_transform(trainPredict)
true_pre1=scaler.inverse_transform(trainPredict1)
true_pre2=scaler.inverse_transform(trainPredict2)
true_pre3=scaler.inverse_transform(trainPredict3)


In [ ]:
fig=plt.figure(figsize=(15,5))
plt.suptitle('Time-Series Prediction')
plt.plot(range(len(L_true)),true_pre[n_input+Q:n_input+Q+N],label='Prediction global model 0',c="green")
plt.plot(range(len(L_true)),true_pre1[n_input+Q:n_input+Q+N],label='Prediction global model 1',c="yellow")
plt.plot(range(len(L_true)),true_pre2[n_input+Q:n_input+Q+N],label='Prediction global model 2',c="blue")
plt.plot(range(len(L_true)),true_pre3[n_input+Q:n_input+Q+N],label='Prediction global model 3',c="purple")
plt.plot(range(len(L_true)),L_true,label='True value',c="red")
plt.legend()
plt.show()

In [ ]:
import json

In [ ]:
def create_Model(n_input,activa, LSTM_neurons = 100, layers=3,n_features=0):
  # define model
  model = Sequential()
  if(layers>1):
    model.add(LSTM(LSTM_neurons, activation=activa, input_shape=(n_input, n_features+1),return_sequences=True))
    for i in range(layers-2):
        model.add(LSTM(LSTM_neurons, activation=activa,return_sequences=True))
    model.add(LSTM(LSTM_neurons, activation=activa))
  else:
    model.add(LSTM(LSTM_neurons, activation=activa, input_shape=(n_input, n_features+1)))
  model.add(Dense(1))
  model.compile(optimizer='adam', loss='mse')
  return model

In [ ]:
activa=["relu","elu","exponential","sigmoid","tanh","softsign"]
min_RMSE=10
for activa1 in activa:
  for Back_samples in range(18,28,2): #hasta dos días y medio hacia el pasado
    for number_neurons in range(50,126,25): 
      for batch_size in range(50,201,50):
        for num_layers in range(1,4): 
          #if((Back_samples,activa,number_neurons,batch_size,num_layers) in combinaciones_anteriores):
           # continue
          print({
              'Retrasos':Back_samples,
              'Activation':activa1,
              'Neuronas LSTM':number_neurons,
              'Batch size':batch_size,
              'LSTM Layers': num_layers
          })
          n_input = Back_samples # horas hacia atrás
          n_output = 1
          n_features = 0 # variables exógenas
          generator = TimeseriesGenerator(scaled_1_train, scaled_1_train, length=n_input, batch_size=n_output)
          model = create_Model(n_input,activa1,LSTM_neurons=number_neurons,layers=num_layers,n_features=n_features)
          model.fit(generator,epochs=7,verbose=False,batch_size=batch_size)
          predictions = []
          for j in range(n_input,len(list(test_data_1.iloc[:,2]))):
            model_input = scaled_1_test[j-n_input:j]
            model_input = model_input.reshape((1, n_input, n_features+1))
            predictions.append(model.predict(model_input)[0])

          predictions = np.ndarray.flatten(np.array(predictions))  
          test_predictions =  np.zeros(shape=(len(predictions), 5))
          test_predictions[:,0] = predictions
          true_predictions = scaler.inverse_transform(test_predictions)
          rsg_predictions = true_predictions[:,0]
          true_values = np.ndarray.flatten(test_data_1.iloc[n_input:][mesure[2]].values)

          errorMSE = mean_squared_error(true_values,rsg_predictions)
          errorMAE = mean_absolute_error(true_values,rsg_predictions)
          errorRMSE = np.sqrt(errorMSE)
          if(errorRMSE<min_RMSE):
            print('BEST')
            model.save('./drive/MyDrive/DatosInvestigacion/LSTM_1_RS.h5')
            min_RMSE = errorRMSE

          print('RMSE:',errorRMSE)
          print('MAE',errorMAE)
          print('min RMSE',min_RMSE)
          resultado = {
              'Retrasos':Back_samples,
              'Activation':activa1,
              'Neuronas LSTM':number_neurons,
              'Batch size':batch_size,
              'LSTM Layers': num_layers,
              'MSE':errorMSE,
              'MAE':errorMAE,
              'RMSE':errorRMSE
          }
          with open("./drive/MyDrive/DatosInvestigacion/resultados_LSTM_1_RS.txt","a") as file:
            resultado = json.dumps(resultado)
            file.write(resultado)
            file.write('\n')